# Get XyXy locations

#### Load Python tools and Jupyter config

In [22]:
import us
import json
import requests
import pandas as pd
import jupyter_black
import altair as alt
import geopandas as gpd
from bs4 import BeautifulSoup
from vega_datasets import data
import xml.etree.ElementTree as ET
from tqdm.notebook import tqdm, trange

In [27]:
jupyter_black.load()
pd.options.display.max_columns = 100
pd.options.display.max_rows = 1000
pd.options.display.max_colwidth = None

In [ ]:
headers = {
    "Accept": "text/javascript, text/html, application/xml, text/xml, */*",
    "Referer": "https://hosted.where2getit.com/dollartree/",
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36",
    "sec-ch-ua-platform": '"macOS"',
}

response = requests.get(
    "https://hosted.where2getit.com/dollartree/ajax?&xml_request=%3Crequest%3E%3Cappkey%3E134E9A7A-AB8F-11E3-80DE-744E58203F82%3C%2Fappkey%3E%3Cformdata+id%3D%22locatorsearch%22%3E%3Cdataview%3Estore_default%3C%2Fdataview%3E%3Climit%3E250%3C%2Flimit%3E%3Cgeolocs%3E%3Cgeoloc%3E%3Caddressline%3E90066%3C%2Faddressline%3E%3Clongitude%3E-118.4338345%3C%2Flongitude%3E%3Clatitude%3E34.0050173%3C%2Flatitude%3E%3C%2Fgeoloc%3E%3C%2Fgeolocs%3E%3Csearchradius%3E100%3C%2Fsearchradius%3E%3Cwhere%3E%3Cicon%3E%3Ceq%3E%3C%2Feq%3E%3C%2Ficon%3E%3Cebt%3E%3Ceq%3E%3C%2Feq%3E%3C%2Febt%3E%3Ccrafters_square%3E%3Ceq%3E%3C%2Feq%3E%3C%2Fcrafters_square%3E%3Cfreezers%3E%3Ceq%3E%3C%2Feq%3E%3C%2Ffreezers%3E%3Csnack_zone%3E%3Ceq%3E%3C%2Feq%3E%3C%2Fsnack_zone%3E%3C%2Fwhere%3E%3C%2Fformdata%3E%3C%2Frequest%3E",
    headers=headers,
)

In [62]:
# Assuming response.text contains your XML data
xml_data = response.text

# Parse the XML
tree = ET.ElementTree(ET.fromstring(xml_data))
root = tree.getroot()

In [63]:
collection_metadata = {}
if root.tag == "response" and root.attrib.get("code") == "1":
    collection = root.find("collection")
    if collection is not None:
        collection_metadata = collection.attrib

In [64]:
poi_data = []

for poi in collection.findall("poi"):
    poi_info = {child.tag: child.text for child in poi}
    # Include collection metadata with each poi data if needed
    # for key, value in collection_metadata.items():
    #     poi_info[f'collection_{key}'] = value
    poi_data.append(poi_info)

# Convert to DataFrame
df_pois = pd.DataFrame(poi_data)

In [65]:
# Example: Convert latitude and longitude to floats
df_pois["latitude"] = pd.to_numeric(df_pois["latitude"], errors="coerce")
df_pois["longitude"] = pd.to_numeric(df_pois["longitude"], errors="coerce")

# Fill NaNs if needed
df_pois.fillna(value={"some_column": "default_value"}, inplace=True)

In [66]:
df_pois[
    [
        "name",
        "address1",
        "address2",
        "city",
        "clientkey",
        "latitude",
        "longitude",
        "phone",
        "postalcode",
        "state",
        "status",
        "temp_closed",
        "timezone",
        "uid",
    ]
]

,name,address1,address2,city,clientkey,latitude,longitude,phone,postalcode,state,status,temp_closed,timezone,uid
0,DollarTree,11455 Jefferson Blvd.,Jefferson Boulevard Shopping Center,Culver City,4187,33.989584,-118.396226,310-754-3559,90230-6105,CA,None,None,America/Los_Angeles,1297376571
1,DollarTree,10561 West Pico Boulevar,10561 West Pico Boulevard,Los Angeles,9725,34.044451,-118.420917,213-838-0504,90064-2319,CA,None,None,America/Los_Angeles,925142934
2,DollarTree,1837 S La Cienega Blvd. Ste A,S La Cienega LA,Los Angeles,7879,34.043403,-118.378852,424-326-7991,90035-4666,CA,None,0,America/Los_Angeles,329804361
3,DollarTree,3710 S La Brea Avenue Unit A,La Brea,Los Angeles,5013,34.019356,-118.355598,323-200-2564,90016-5310,CA,None,0,America/Los_Angeles,1297377506
4,DollarTree,811 N La Brea Ave,Inglewood,Inglewood,4995,33.973811,-118.357631,310-981-5082,90302-3641,CA,None,0,America/Los_Angeles,1297377485
5,DollarTree,5057 W. Washington Blvd,Washington Property,Los Angeles,5595,34.041041,-118.349973,323-761-6090,90016-1450,CA,None,0,America/Los_Angeles,1297378160
6,DollarTree,3828 W Slauson Ave.,F/S Slauson,Los Angeles,6262,33.988970,-118.340899,323-200-2566,90043-2935,CA,None,None,America/Los_Angeles,605585044
7,DollarTree,4747 W Century Blvd,Inglewood,Inglewood,4860,33.945440,-118.360148,310-981-5062,90304-1441,CA,None,0,America/Los_Angeles,1297377331
8,DollarTree,11307 Hawthorne Blvd .,F/S Hawthorne Blvd,Inglewood,6233,33.931900,-118.352710,310-242-2185,90304-2714,CA,None,None,America/Los_Angeles,171014581
9,DollarTree,2542 W Manchester Blvd,Former Goodyear Bldg,Inglewood,7007,33.959708,-118.321494,323-781-2032,90305-2520,CA,None,None,America/Los_Angeles,329803470
